In [150]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [151]:
print(os.listdir())

['.conda', '.git', '.gitattributes', '.gitignore', '1 LEGO_Datasets.ipynb', 'colors.csv', 'demo.py', 'downloads_schema.png', 'inventories.csv', 'inventory_parts.csv', 'inventory_sets.csv', 'LICENSE.txt', 'markdown-diff', 'morelearning.ipynb', 'parts.csv', 'part_categories.csv', 'README.md', 'sets.csv', 'themes.csv']


## Creating the Panda dataframes

In [152]:
colors = pd.read_csv("colors.csv", dtype={'name': 'str', 'rgb': 'str'})
inventories = pd.read_csv("inventories.csv")
inventory_parts = pd.read_csv("inventory_parts.csv")
inventory_sets = pd.read_csv("inventory_sets.csv")
part_categories = pd.read_csv("part_categories.csv")
parts = pd.read_csv("parts.csv")
sets = pd.read_csv("sets.csv")
themes = pd.read_csv("themes.csv")

## Creating the investigate function

In [153]:
def investigate(db):
    """explores the data"""

    
    print(f"Exploring the data in the dataframe")
    print(db.index)
    print("")
    print("COLUMN NAMES")
    print(db.columns)
    print()
    nRow, nCol = db.shape
    print(f'There are {nRow} rows and {nCol} columns')
    print()
    print("DATA TYPES")
    print(db.dtypes)
    print()
    print("NUMBER OF UNIQUE VALUES IN EACH COLUMN")
    print(db.nunique())
    print()
    print("PREVIEWING THE DATASET")
    print(db.head())
    print()
    print(db.info())
    print()
    print("NUMBER OF NON-NULL VALUES IN EACH COLUMN")
    print(db.count())


In [154]:
# Remove extraneous quotes
colors['name'] = colors['name'].str.replace(r'^["\']|["\']$', '', regex=True)
colors['name'] = colors['name'].str.replace(r"'-", '', regex=True)
colors['name'] = colors['name'].str.replace(r'[\[\]]', '', regex=True) #remove brackets
# Strip leading/trailing spaces
colors['name'] = colors['name'].str.strip()
# Replace 'f' with False and 't' with True
colors['is_trans'] = colors['is_trans'].replace({'f': False, 't': True})
# Define the mapping of color names to groups
color_groups = {
    'Red': 'Red',
    'Green': 'Green',
    'Blue': 'Blue',
    'Light Blue': 'Blue',
    'Dark Green': 'Green',
    'Medium Lavender': 'Purple',
    'Yellow': 'Yellow',
    'Orange': 'Orange',
    'Pink': 'Pink',
    'Unknown': 'Unknown',
    'Black': 'Black',
    'Dark Turquoise': 'Blue',
    'Medium Blue': 'Blue',
    'Tan': 'Brown',
    'Light Gray': 'Gray',
    'Dark Gray': 'Gray',
    'Brown': 'Brown',
    'White': 'White',
    'Purple': 'Purple',
    'Lime': 'Green',
    'Magenta': 'Pink',
    'Dark Pink': 'Pink',
    'Light Pink': 'Pink',
    'Bright Pink': 'Pink',
    'Lavender': 'Purple',
    'Sand Blue': 'Blue',
    'Sand Green': 'Green',
    'Sand Purple': 'Purple',
    'Sand Red': 'Red',
    'Sand Yellow': 'Yellow',
    'Dark Red': 'Red',
    'Medium Dark Pink': 'Pink',
    'Dark Brown': 'Brown',
    'Medium Azure': 'Blue',
    'Medium Blue': 'Blue',
    'Medium Green': 'Green',
    'Medium Lime': 'Green',
    'Medium Orange': 'Orange',
    'Medium Red': 'Red',
    'Medium Violet': 'Purple',
    'Medium Yellow': 'Yellow',
    'Bright Light Blue': 'Blue',
    'Bright Light Orange': 'Orange',
    'Bright Light Yellow': 'Yellow',
    'Bright Purple': 'Purple',
    'Bright Red': 'Red',
    'Bright Yellow': 'Yellow',
    'Light Aqua': 'Blue',
    'Light Green': 'Green',
    'Light Lime': 'Green',
    'Light Orange': 'Orange',
    'Light Purple': 'Purple',
    'Light Salmon': 'Red',
    'Light Turquoise': 'Blue',
    'Light Yellow': 'Yellow',
    'Olive Green': 'Green',
    'Very Light Bluish Gray': 'Gray',
    'Very Light Gray': 'Gray',
    'Very Light Orange': 'Orange',
    'Very Light Yellow': 'Yellow',
    'Dark Blue-Violet': 'Blue',
    'Dark Blue': 'Blue',
    'Dark Azure': 'Blue',
    'Dark Orange': 'Orange',
    'Dark Tan': 'Brown',
    'Dark Purple': 'Purple',
    'Dark Flesh': 'Brown',
    'Dark Salmon': 'Red',
    'Dark Yellow': 'Yellow',
    'Light Blue-Violet': 'Blue',
    'Light Bluish Gray': 'Gray',
    'Light Grayish Blue': 'Gray',
    'Light Grayish Orange': 'Orange',
    'Light Nougat': 'Brown',
    'Light Purple': 'Purple',
    'Light Salmon': 'Red',
    'Light Yellowish Orange': 'Yellow',
    'Medium Blue-Violet': 'Blue',
    'Medium Bluish Gray': 'Gray',
    'Medium Dark Flesh': 'Brown',
    'Medium Dark Pink': 'Pink',
    'Medium Lavender': 'Purple',
    'Medium Nougat': 'Brown',
    'Medium Purple': 'Purple',
    'Medium Reddish Brown': 'Brown',
    'Medium Violet': 'Purple',
    'Medium Yellowish Orange': 'Yellow',
    'Bright Bluish Green': 'Green',
    'Bright Bluish Violet': 'Blue',
    'Bright Green': 'Green',
    'Bright Orange': 'Orange',
    'Bright Purple': 'Purple',
    'Bright Reddish Violet': 'Red',
    'Bright Violet': 'Purple',
    'Bright Yellowish Green': 'Yellow',
    'Light Bluish Green': 'Green',
    'Light Bluish Violet': 'Blue',
    'Light Bright Blue': 'Blue',
    'Light Bright Green': 'Green',
    'Light Brown': 'Brown',
    'Trans-Yellow': 'Yellow',
    'Trans-Black IR Lens': 'Black',
    'Light': 'White',
    'Salmon': 'Red',
    'Light Violet': 'Purple',
    'Glow In Dark Opaque': 'White',
    'Dark Violet' : 'Purple',
    'Trans Flame Yellowish Orange': 'Orange',
    'Trans Fire Yellow': 'Yellow',
    'Trans Light Royal' : 'Blue',
    'Reddish Lilac': 'Purple',
    'No Color' : 'Clear',
    'Trans-Dark': 'Black',
    'Trans-Green': 'Green',
    'Trans-Bright Green': 'Green',
    'Trans-Red': 'Red',
    'Trans-Black': 'Black',
    'Chrome Gold': 'Metallic',
    'Chrome Antique Brass': 'Metallic',
    'Chrome Black': 'Metallic',
    'Chrome Blue': 'Metallic',
    'Chrome Green': 'Metallic',
    'Chrome Pink': 'Metallic',
    'Chrome Silver': 'Metallic',
    'Fabuland Brown': 'Brown',
    'Glow in Dark White': 'White',
    'Trans-Light' : 'Clear',
    'Trans-Neon Green': 'Green',
    'Trans-Very Lt': 'Clear',
    'Trans-Dark Pink': 'Pink',
    'Trans-Clear': 'Clear',
    'Trans-Purple': 'Purple',
    'Trans-Neon Orange': 'Orange',
    'Chrome' : 'Metallic',
    'Reddish Brown': 'Brown',
    'Trans-Neon Yellow': 'Yellow',
    'Dark Bluish Gray': 'Gray',
    'Medium': 'Unknown',
    'Speckle Black-Copper': 'Metallic',
    'Speckle DBGray-Silver': 'Metallic',
    'Light Flesh': 'Pink',
    'Milky White': 'White',
    'Metallic Silver': 'Metallic',
    'Metallic Gold': 'Metallic',
    'Metallic Green': 'Metallic',
    'Metallic Gold' : 'Metallic',
    'Royal': 'Blue',
    'Flesh': 'Pink',
    'Violet': 'Purple',
    '-Violet': 'Purple',
    'Glitter Trans-Dark Pink': 'Pink',
    'Glitter Trans-Clear': 'Clear',
    'Aqua': 'Blue',
    
    


    
}

# Apply the mapping to create a new column 'group'
colors['group'] = colors['name'].map(color_groups)

# Count the number of unique names
unique_groups_count = colors['group'].nunique()
# Display the count of unique names
print(f"Number of unique groups: {unique_groups_count}")
print()

# Check for unassigned names
unassigned_names = colors[colors['group'].isna()]

# Count how many have group as NaN
na_count = unassigned_names.shape[0]
print(f"Number of unassigned names : {na_count}")
print()

# Display unassigned names
if not unassigned_names.empty:
    print("Unassigned color names:")
    print(unassigned_names[['id', 'name']])
else:
    print("All color names have been assigned to a group.")

# Count the number of color names containing 'glitter'
glitter_count = colors[colors['name'].str.contains('glitter', case=False, na=False)].shape[0]

# Display the count of color names containing 'glitter'
print(f"Number of color names containing 'glitter': {glitter_count}")
print()

print(colors.head())

Number of unique groups: 14

Number of unassigned names : 28

Unassigned color names:
       id                      name
80    129      Glitter Trans-Purple
81    132      Speckle Black-Silver
82    133        Speckle Black-Gold
83    134                    Copper
84    135          Pearl Light Gray
85    137                     Metal
86    142          Pearl Light Gold
87    143              Trans-Medium
88    148           Pearl Dark Gray
89    150     Pearl Very Light Gray
91    158           Yellowish Green
92    178            Flat Dark Gold
93    179               Flat Silver
94    182              Trans-Orange
95    183               Pearl White
97    212              Bright Light
98    216                      Rust
100   230                Trans-Pink
101   232                       Sky
102   236        Trans-Light Purple
103   272                      Dark
105   294        Glow In Dark Trans
106   297                Pearl Gold
108   313                    Maersk
117   366     

## Using RGB hex codes for color names

In [156]:
import webcolors

def hex_to_color_name(hex_value):
    try:
        # Convert hex to RGB tuple
        rgb_value = webcolors.hex_to_rgb(hex_value)
        # Get the closest color name
        color_name = webcolors.rgb_to_name(rgb_value)
    except ValueError:
        # If the exact color name is not found, find the closest match
        color_name = closest_color_name(rgb_value)
    return color_name

def closest_color_name(rgb_value):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - rgb_value.red) ** 2
        gd = (g_c - rgb_value.green) ** 2
        bd = (b_c - rgb_value.blue) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

# Create a new column 'color_name' to store the standard color names
colors['color_name'] = colors['rgb'].apply(hex_to_color_name)

print(colors.head())

UnboundLocalError: local variable 'rgb_value' referenced before assignment

# Ideas for this data set
* Group colors into 10 groups (124 colors out of 135 is too many)
* Track popular themes and colors by year
    ** datasets= colors, sets, themes
* Only 12 pieces had more than version 1 released.  Where those the popular colors and themes?
    ** dataset = inventories
* How has the size of sets changed over time?
* What colors are associated with which themes? 
* Could you predict which theme a set is from just by the bricks it contains?
* What sets have the most-used pieces in them? What sets have the rarest pieces in them?
* Have the colors of LEGOs included in sets changed over time?



# Exploring each dataset

## colors dataset

In [ ]:
investigate(colors)

### What's with the color names?

In [ ]:
colors['name'].unique()

### rgb values

In [ ]:
uniquecolors = colors['rgb'].nunique()
nRow, nCol = colors.shape

print(f"There are {uniquecolors} unique colors in the dataframe colors out of {nRow} total colors")
print()
print("COUNT OF ALL UNIQUE COLORS")
colors['rgb'].value_counts()


## inventory_sets dataset

The dataset inventory_sets has 3 columns: inventory_id, set_num and quauntity.  The inventory_id is the piece/part, the quantity is the number of a specfic piece (inventory_id) included.  A set_num has multiple pieces (inventory_id).  A piece (inventory_id in inventory_sets = id in inventories) in some cases has had different versions (the versions of a piece are in the inventories dataset)

The color is available for each inventory_id, and each inventory_id is categorized (windows, doors, bricks).

In [ ]:
investigate(inventory_sets)

#### Most sets have only 1 of any kind of piece in the set.  There are 2,846 total peices (count of inventory_id).  2,793 pieces only occur once in a set (see below).

To do: filter using Python to find what iventory_ids have quantities over 1.  What set_nums are they part of, what themes are they part of?

In [ ]:
print("COUNT OF quantities...quantity and the number with that quantity")
inventory_sets['quantity'].value_counts()

quantity= the number of that iventory_id were included.  Each set_num has meany inventory_ids.


What does it mean that quantity was "1" for most sets?  What does it mean if there was more than 1 of a set? the set "Basic Building Set" with set_num = 044-1 had what quantity?  This can't be the number sold, because the highest number is 60.

044-1,Basic Building Set,1968,366,225

set_num 4520-1 had quantity = 2
name = Curved Rails
year = 1991
theme_id =244
num_parts = 8

14 sets had theme 244

set_num = 4520-1, Curved Rails, 1991, theme = 244, 8 parts
set_num = 4531-1, Manual Points, 1991, theme= 244, 6 parts

set num 4530-1 had quantity 2 of iventory_id 50, quantity 5 of inventory_id 309, quantity 1 of inventory 6050, etc.


## inventories dataset

In [ ]:
investigate(inventories)

#### Very few sets had more than one version released

In [ ]:
print("COUNT OF DIFFERENT VERSIONS")
inventories['version'].value_counts()

## themes dataset

In [ ]:
investigate(themes)

### 78 total themes

In [ ]:
nthemegroups = themes.parent_id.nunique()
print(f"There are {nthemegroups} total themes.")

## sets dataset

In [ ]:
investigate(sets)

In [ ]:
inventories.set_num.value_counts

## sets part_categories

In [ ]:
investigate(part_categories)